# Corner simulation 

In this demonstration, we'll construct a **corner** situation and simulate the evacuation of **5 agents** positioned on a grid.

In buildings, corners are inevitable. It is important to simulate this kind of geometry in order to understand how could people behave and react. 

Let's begin by importing the required packages for our simulation:

In [ ]:
from shapely import GeometryCollection, Polygon, to_wkt
import pathlib
import jupedsim as jps
import pedpy
import pandas as pd
import numpy as np
import plotly  # visualise trajectories
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import Figure
import sqlite3

%matplotlib inline

## Setting up a geometry

In [ ]:
p1 = Polygon([(0, 0), (30, 0), (30, 10), (0, 10)])
p2 = Polygon([(22, 10), (30, 10), (30, 30), (22, 30)])
area = GeometryCollection(p1.union(p2))
area

## Operational model
<a id="model"></a>
Once the geometry is set, our subsequent task is to specify the model and its associated parameters.
For this demonstration, we'll employ the "collision-free" model.

## Setting Up the Simulation Object

Having established the model and geometry details, and combined with other parameters such as the time step dt, we can proceed to construct our simulation object as illustrated below:

In [ ]:
trajectory_file = "corner.sqlite"
simulation = jps.Simulation(
    model=jps.VelocityModelParameters(),
    geometry=area,
    trajectory_writer=jps.SqliteTrajectoryWriter(
        output_file=pathlib.Path(trajectory_file)
    ),
)

## Specifying Routing Details

At this juncture, we'll provide basic routing instructions, guiding the agents to progress towards the exit point.

In [ ]:
exit_id = simulation.add_exit_stage([(22, 25), (30, 25), (30, 30), (22, 30)])
journey = jps.JourneyDescription([exit_id])
journey_id = simulation.add_journey(journey)

## Defining and Distributing Agents

Now, we'll position the agents and establish their attributes, leveraging previously mentioned parameters such as `exit_id` and `profile_id`.

In [ ]:
agent_parameters = jps.VelocityModelAgentParameters()
agent_parameters.journey_id = journey_id
agent_parameters.stage_id = exit_id
agent_parameters.v0 = 1.2
agent_parameters.radius = 0.15
agent_parameters.time_gap = 1

for new_pos in [(7, 7), (1, 3), (1, 5), (1, 7), (2, 7)]:
    agent_parameters.position = new_pos
    simulation.add_agent(agent_parameters)

## Executing the Simulation

With all components in place, we're set to initiate the simulation.
For this demonstration, the trajectories will be recorded in an sqlite database.

In [ ]:
while simulation.agent_count() > 0:
    simulation.iterate()

## Visualizing the Trajectories

For trajectory visualization, we'll extract data from the sqlite database. A straightforward method for this is employing the jupedsim-visualizer.

To-Do List:

    Incorporate references and hyperlinks to additional resources.
    Integrate results visualization using the visualizer.

## References & Further Exploration

The operational model discussed in the Model section is based on the collision-free model. JuPedSim also incorporates another model known as GCFM. For more details on GCFM, refer to another notebook (TODO: Link to the GCFM notebook).

Our demonstration employed a straightforward journey with a singular exit. For a more intricate journey featuring multiple intermediate stops and waiting zones, see the upcoming section (TODO: Link to the advanced journey section).

While we designated a single parameter profile for agents in this example, it's feasible to define multiple parameter profiles. Learn how to alternate between these profiles in the subsequent section (TODO: Link to the profile-switching section).

In [ ]:
def read_trajectory_data(trajectory_file: str) -> pedpy.TrajectoryData:
    with sqlite3.connect(trajectory_file) as con:
        data = pd.read_sql_query(
            "select frame, id, pos_x as x, pos_y as y, ori_x as ox, ori_y as oy from trajectory_data",
            con,
        )
        fps = float(
            con.cursor()
            .execute("select value from metadata where key = 'fps'")
            .fetchone()[0]
        )
        return pedpy.TrajectoryData(data=data, frame_rate=fps)


def moving_agents(data_df: pd.DataFrame, area: GeometryCollection) -> Figure:
    """Generate moving trajectories based on simulation"""

    data_df["radius"] = 0.2
    fig = px.scatter(
        data_df,
        x="x",
        y="y",
        animation_frame="frame",
        animation_group="id",
        size="radius",
    )
    x, y = area.geoms[0].exterior.xy
    fig.add_trace(
        go.Scatter(
            x=np.array(x),
            y=np.array(y),
            fill="toself",
            mode="lines",
            line={"color": "grey"},
        )
    )

    minx, miny, maxx, maxy = area.bounds
    fig.update_xaxes(range=[minx, maxx])
    fig.update_yaxes(range=[miny, maxy])
    fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 10
    fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 5
    fig.update_geos(
        projection_type="equirectangular", visible=True, resolution=110
    )
    # fig.update_traces(marker={"line": {"width": 0.5, "color": "Gray"}})
    # fig.update_geos(projection_type="equirectangular", visible=True, resolution=110)
    fig.update_layout(showlegend=False)
    return fig

In [ ]:
trajectory_data = read_trajectory_data(trajectory_file)
moving_agents(trajectory_data.data, area)

The agents want to go the shortest way and go directly to the inside corner of the the geometry. The two last ones a